# Data Analysis

Download python libraries using the [pip](https://pip.pypa.io/en/stable/quickstart/) command.

In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import time
import math
from plotnine import *
import numpy as np
import os

In [ ]:
#import data
a1 = pd.read_csv(os.environ['WORKDIR'] + '/kallisto/1a/abundance.tsv', sep='\t', header=0, usecols=['target_id', 'tpm'])
a2 = pd.read_csv(os.environ['WORKDIR'] + '/kallisto/2a/abundance.tsv', sep='\t', header=0, usecols=['target_id', 'tpm'])
a3 = pd.read_csv(os.environ['WORKDIR'] + '/kallisto/3a/abundance.tsv', sep='\t', header=0, usecols=['target_id', 'tpm'])
c1 = pd.read_csv(os.environ['WORKDIR'] + '/kallisto/1c/abundance.tsv', sep='\t', header=0, usecols=['target_id', 'tpm'])
c2 = pd.read_csv(os.environ['WORKDIR'] + '/kallisto/2c/abundance.tsv', sep='\t', header=0, usecols=['target_id', 'tpm'])
c3 = pd.read_csv(os.environ['WORKDIR'] + '/kallisto/3c/abundance.tsv', sep='\t', header=0, usecols=['target_id', 'tpm'])

#parse each sample's data
a1.rename(columns = {'tpm':'a1'}, inplace = True)
a2.rename(columns = {'tpm':'a2'}, inplace = True)
a3.rename(columns = {'tpm':'a3'}, inplace = True)
c1.rename(columns = {'tpm':'c1'}, inplace = True)
c2.rename(columns = {'tpm':'c2'}, inplace = True)
c3.rename(columns = {'tpm':'c3'}, inplace = True)
a2 = a2.drop('target_id', axis=1)
a3 = a3.drop('target_id', axis=1)
c1 = c1.drop('target_id', axis=1)
c2 = c2.drop('target_id', axis=1)
c3 = c3.drop('target_id', axis=1)

#combine samples into one table, where each row represents a sample and columns represent transcript
dataframe = pd.concat([a1,a2,a3,c1,c2,c3], axis=1)
dataframe = dataframe.T
dataframe.columns = dataframe.iloc[0]
dataframe = dataframe.drop('target_id')

#clear memory, don't need references anymore
del a1,a2,a3,c1,c2,c3

#clean data by removing possible noise
#filter out columns where any value is less than 10 (lowly expressed)
min_threshold = 10
for column in dataframe:
    for value in dataframe[column]:
        if value < min_threshold:
            dataframe.drop(column, axis=1, inplace=True)
            break

#log base 10 all values in dataframe
dataframe.applymap(lambda x: math.log(x, 10))

In [ ]:
# perform PCA calculations
pca = PCA(n_components=2)
pca_result = pca.fit_transform(dataframe.values)

print ('Explained variation per principal component {}'.format(pca.explained_variance_ratio_))
dataframe['PC1'] = pca_result[:,0]
dataframe['PC2'] = pca_result[:,1] 
dataframe['label'] = pd.Series(['before', 'before', 'before', 'after', 'after', 'after'], index = dataframe.index)

In [ ]:
# graph t-SNE
chart = ggplot(dataframe, aes(x='tSNE1', y='tSNE2', color='label') ) \
        + geom_point(size=3,alpha=1) \
        + ggtitle("tSNE dimensions colored by digit")
chart